# dlt & SQLMesh Runner

## Install Dependencies

In [ ]:
%pip install "dlt[az,deltalake,filesystem,parquet]>=1.12.1"
%pip install "enlighten>=1.14.1"
%pip install "ibis==3.3.0"
%pip install "networkx>=3.5"
%pip install "sqlmesh[fabric,mssql]"

## Set Environment Vars

In [ ]:
import os
os.environ["DESTINATION__BUCKET_URL"] = "/lakehouse/default/Files/northwind-to-fabric/Tables"

### Load From KeyVault

In [ ]:
import os

keyvault = "mattiasthalen-fabric"

credentials = [
    "CREDENTIALS__AZURE_TENANT_ID",
    "CREDENTIALS__AZURE_CLIENT_ID",
    "CREDENTIALS__AZURE_CLIENT_SECRET",
]

for credential in credentials:
    secret = credential.replace("_", "-")
    value = notebookutils.credentials.getSecret(f"https://{keyvault}.vault.azure.net/", secret)
    os.environ[credential] = value

    print(f"{credential} = {os.getenv(credential)}")

### Load From Workspace

In [ ]:
import sempy.fabric as fabric
import os
import pandas as pd

state__item_name = "sqlmesh__state"
lakehouse__item_name = "landing_zone"

client = fabric.FabricRestClient()

# Get workspace id and items
workspace_id = fabric.get_workspace_id()
workspace__response = client.get(f"/v1/workspaces/{workspace_id}/items")
workspace__items = pd.json_normalize(workspace__response.json()['value'])

# Get state endpoint and database name
state__items = workspace__items[workspace__items["displayName"] == state__item_name]
state__item_id = state__items[state__items["type"] == "SQLDatabase"].iloc[0, 0]

state__response = client.get(f"/v1/workspaces/{workspace_id}/sqlDatabases/{state__item_id}")
state__json = state__response.json()
state__database = state__json["properties"]["databaseName"]
state__endpoint = state__json["properties"]["serverFqdn"].split(",")[0]

# Get warehouse endpoint
warehouse__endpoint = state__endpoint.replace(".database.", ".datawarehouse.")

# Set env vars
vars = {
    "FABRIC__WORKSPACE_ID": workspace_id,
    "FABRIC__WAREHOUSE_ENDPOINT": warehouse__endpoint,
    "FABRIC__STATE_ENDPOINT": state__endpoint,
    "FABRIC__STATE_DATABASE": state__database,
}

for name, value in vars.items():
    if not value:
        print(f"Null value for {name}")
    
    if value:
        os.environ[name] = value
        print(f"Value set for {name}")

## Sync Codebase

In [ ]:
import git

organization = "mattiasthalen"
repo_name = "northwind-to-fabric"

url = f"https://github.com/{organization}/{repo_name}.git"

branch = "main"
code_path = f"/lakehouse/default/Files/{repo_name}"

try:
    repo = git.Repo.clone_from(
        url=url,
        to_path=code_path,
        branch=branch,
        depth=1,
        single_branch=True
    )

    print(f"Cloned repo to {code_path}")

except git.GitCommandError as e:
    if e.status == 128:
        repo = git.Repo(code_path)
        origin = repo.remotes.origin
        repo.git.reset('--hard', f'origin/{branch}')
        origin.pull(branch)

        print("Pulled latest changes from repo")
        
    else:
        raise

## Run Project

### Run dlt

In [ ]:
import os
import subprocess

dlt_path = os.path.join(code_path, "dlt")

cmd = ["python", "northwind.py", "prod"]
result = subprocess.run(
    cmd,
    cwd=dlt_path,
    text=True,
    check=True
) 

### Run SQLMesh

In [ ]:
import os
import subprocess

sqlmesh_path = os.path.join(code_path, "sqlmesh")

cmd = ["sqlmesh", "plan", "prod", "--run", "--auto-apply", "--no-prompts"]
result = subprocess.run(
    cmd,
    cwd=sqlmesh_path,
    text=True,
    check=True
) 